In [9]:
path = '~/.ros/tracing/canopen-controller-test9'

In [10]:
import os
import sys
import datetime as dt
from typing import List, Optional
from typing import Tuple
from typing import Union

import numpy as np
import pandas as pd

dir = '.' # os.path.dirname(os.path.realpath(__file__))
sys.path.insert(0, os.path.join(dir, '../../tracetools_analysis/tracetools_analysis'))
sys.path.insert(0, os.path.join(dir, '../../ros2_tracing/tracetools_read'))
from tracetools_analysis.loading import load_file
from tracetools_analysis.processor.ros2 import Ros2Handler
from tracetools_analysis.utils.ros2 import Ros2DataModelUtil

In [11]:
events = load_file(path)
handler = Ros2Handler.process(events)

converting trace directory: /home/admin/.ros/tracing/canopen-controller-test9
converted 660769 events in 42.6 s
output written to: /home/admin/.ros/tracing/canopen-controller-test9/converted
 [100%] [Ros2Handler]


In [12]:
data_util = Ros2DataModelUtil(handler.data)

callback_symbols = data_util.get_callback_symbols()

In [13]:
tpdo_data = data_util.data.canopen_tpdo_data
type(tpdo_data)

pandas.core.frame.DataFrame

In [14]:
tpdo_data

""


In [15]:
tpdo_data = tpdo_data.drop('node_id', axis=1)
tpdo_data['timestamp'] = pd.to_datetime(tpdo_data['timestamp'], unit='ns')
tpdo_data['time_increment'] = (tpdo_data['timestamp'] - tpdo_data['timestamp'].iloc[0]).dt.total_seconds()

KeyError: "['node_id'] not found in axis"

In [ ]:
divide_by_constant = lambda x: x/((180/np.pi)*1000)
tpdo_data['data'] = tpdo_data['data'].apply(divide_by_constant)

In [ ]:
tpdo_prbt_joint_1 = tpdo_data.loc[tpdo_data['name'].str.contains('prbt_joint_1')]
tpdo_prbt_joint_2 = tpdo_data.loc[tpdo_data['name'].str.contains('prbt_joint_2')]
tpdo_prbt_joint_3 = tpdo_data.loc[tpdo_data['name'].str.contains('prbt_joint_3')]
tpdo_prbt_joint_4 = tpdo_data.loc[tpdo_data['name'].str.contains('prbt_joint_4')]
tpdo_prbt_joint_5 = tpdo_data.loc[tpdo_data['name'].str.contains('prbt_joint_5')]
tpdo_prbt_joint_6 = tpdo_data.loc[tpdo_data['name'].str.contains('prbt_joint_6')]

In [ ]:
time_diff = np.array(tpdo_prbt_joint_2['time_increment']).astype(np.float64)
time_diff = time_diff[1:]

# calculate the expected time increment
time_increment = np.mean(tpdo_prbt_joint_1['time_increment'])

print(f'mean time increment: {time_increment}')

# check if any time differences are greater than the expected time increment
if any(time_diff > 14):
    print('Data skip detected!')
else:
    print('No data skip detected.')

num_skips = np.sum(time_diff > 14)

print(f'Total number of time skips: {num_skips}')
print(f'Max time difference: {np.max(time_diff)}')